In [1]:
import glob
import functools
import datetime as dt
import pandas as pd
import numpy as np
# import h2o
import matplotlib.pyplot as plt
# import shap
from pandas_profiling import ProfileReport
from collections import defaultdict
import seaborn as sns
import os
import random

### 원본데이터를 가져옵니다

In [15]:
dataprep_df_full = pd.read_csv(r'C:\Users\C07037\Python_project\data_Pool\raw_data.csv', delimiter = '\t', encoding='utf-8')
dataprep_df_full.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 56410 entries, 0 to 56409
Data columns (total 25 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   기준일자        56410 non-null  int64  
 1   실행번호        56410 non-null  object 
 2   고객번호        56410 non-null  object 
 3   실행사유코드명     56410 non-null  object 
 4   상품대분류명      56410 non-null  object 
 5   상품중분류명      56410 non-null  object 
 6   상품소분류명      56410 non-null  object 
 7   상품명         56410 non-null  object 
 8   실행일자        56410 non-null  int64  
 9   만기일         56410 non-null  int64  
 10  여신잔액        56410 non-null  float64
 11  순대출금액       56410 non-null  float64
 12  여신기간        56410 non-null  int64  
 13  원리금연체일수     113 non-null    float64
 14  연체대상원금      129 non-null    float64
 15  연체대상이자      129 non-null    float64
 16  마감일자        30872 non-null  float64
 17  해지일자        181 non-null    float64
 18  해지사유        181 non-null    object 
 19  영업부서코드명     56410 non-nul

### 상품명 리스트를 체크 합니다.

In [17]:
dataprep_df_full['상품명'].unique()

array(['신차운용리스', '스탁론(국내)', '오토론신차대출', '임차보증금대출', '스마트스토어대출', '신차금융리스',
       'PF대출', '오토론중고차대출', '오토재고금융대출', '신차할부금융', '중도금대출', '미트론',
       '더존연계팩토링', '스탁론(해외)', '신차오토렌탈', '대부업체대출', '기업자금대출', '바이크운용리스',
       '보통주', 'NPL대출', '오토자금대출', '주식담보대출', '부동산집합투자증권', '단기렌터카(보험대차)',
       '생명연계대출', 'PEF출자금', '증권집합투자증권(혼합형)', '전문투자형사모투자신탁', '투자조합출자금',
       'CB', '기타담보대출', 'BW', '우선주', '증권집합투자증권(주식형)', '기업자금대출(베트남FC)',
       '전환우선주', 'ETF', '회사채', '상환전환우선주', '증권집합투자증권(채권형)'], dtype=object)

### 상기 상품명을 보고 Grouping 할 것을 묶습니다.

In [18]:
auto_group = ['신차운용리스', '오토론신차대출', '신차금융리스', '오토론중고차대출', '오토재고금융대출', '신차할부금융', '신차오토렌탈', '바이크운용리스', '오토자금대출', '단기렌터카(보험대차)' ]
apt_lease = ['임차보증금대출']
stock_loan = ['스탁론(해외)','스탁론(국내)' ]
naver_sme = ['스마트스토어대출']
data_etc = ['더존연계팩토링', '주식담보대출', '미트론', '기타담보대출', '생명연계대출', 'NPL대출', '대부업체대출']
ib_etc = ['PF대출', '기업자금대출', '보통주', '부동산집합투자증권','PEF출자금', '증권집합투자증권(혼합형)', '전문투자형사모투자신탁', '투자조합출자금','CB', 'BW', '우선주', '증권집합투자증권(주식형)', '기업자금대출(베트남FC)', '전환우선주', 'ETF', '회사채', '상환전환우선주', '증권집합투자증권(채권형)' ]

### 위에 정의한 상품별로 Grouping할 함수(Function)를 정의합니다.

In [19]:
def get_product_group(x) :
    if x in auto_group :
        return 'AUTO'
    elif x in apt_lease :
        return '임대보증금'
    elif x in stock_loan :
        return '스탁론'
    elif x in naver_sme :
        return '네이버사업자대출'
    elif x in data_etc :
        return '데이터금융_기타'
    elif x in ib_etc :
        return '투자금융 등'
    else : return '기타'

### 상품명을 상기 Grouping 하는 함수를 적용해서 '상품구분'이라는 컬럼에 추가합니다.

In [20]:
dataprep_df_full['상품구분'] = dataprep_df_full['상품명'].apply(get_product_group)

### 월별 Vintage분석을 위해 실행일자(YYYYMMDD)를 'YYYYMM'으로 변경하고 새로운 컬럼에 추가합니다.

In [21]:
dataprep_df_full['VIN_M'] = dataprep_df_full['실행일자'].apply(lambda x  : str(x)[0:6])

### 각 상품구분별로 데이터 프로파일링 하는 함수를 만듭니다. EDA 과정입니다.
각 상품구분별로 기초통계자료 뽑고 결과를 html 파일로 저장하는 함수입니다.

In [11]:
def make_profile(df) :
    product_name = df['상품구분'].unique()[0]
    profile = df.profile_report(
        variables={
            "descriptions": {
                "files": "Files in the filesystem",
                "datec": "Creation date",
                "datem": "Modification date",
            }
        }
    )

    profile.to_file(r'../output/report_{}.html'.format(product_name))

In [13]:
for prod in dataprep_df_full['상품구분'].unique() :
    df_temp = dataprep_df_full[dataprep_df_full['상품구분'] == prod]
    make_profile(df_temp)

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

C:\anaconda3\envs\tf_nlp\lib\site-packages\pandas_profiling\visualisation\utils.py:66: UserWarning: Glyph 49892 (\N{HANGUL SYLLABLE SIL}) missing from current font.
  plt.savefig(image_str, format=image_format)
C:\anaconda3\envs\tf_nlp\lib\site-packages\pandas_profiling\visualisation\utils.py:66: UserWarning: Glyph 54665 (\N{HANGUL SYLLABLE HAENG}) missing from current font.
  plt.savefig(image_str, format=image_format)
C:\anaconda3\envs\tf_nlp\lib\site-packages\pandas_profiling\visualisation\utils.py:66: UserWarning: Glyph 51068 (\N{HANGUL SYLLABLE IL}) missing from current font.
  plt.savefig(image_str, format=image_format)
C:\anaconda3\envs\tf_nlp\lib\site-packages\pandas_profiling\visualisation\utils.py:66: UserWarning: Glyph 51088 (\N{HANGUL SYLLABLE JA}) missing from current font.
  plt.savefig(image_str, format=image_format)
C:\anaconda3\envs\tf_nlp\lib\site-packages\pandas_profiling\visualisation\utils.py:66: UserWarning: Glyph 47564 (\N{HANGUL SYLLABLE MAN}) missing from curre

C:\anaconda3\envs\tf_nlp\lib\site-packages\pandas_profiling\visualisation\utils.py:66: UserWarning: Glyph 47560 (\N{HANGUL SYLLABLE MA}) missing from current font.
  plt.savefig(image_str, format=image_format)
C:\anaconda3\envs\tf_nlp\lib\site-packages\pandas_profiling\visualisation\utils.py:66: UserWarning: Glyph 44048 (\N{HANGUL SYLLABLE GAM}) missing from current font.
  plt.savefig(image_str, format=image_format)
C:\anaconda3\envs\tf_nlp\lib\site-packages\pandas_profiling\visualisation\utils.py:66: UserWarning: Glyph 51068 (\N{HANGUL SYLLABLE IL}) missing from current font.
  plt.savefig(image_str, format=image_format)
C:\anaconda3\envs\tf_nlp\lib\site-packages\pandas_profiling\visualisation\utils.py:66: UserWarning: Glyph 51088 (\N{HANGUL SYLLABLE JA}) missing from current font.
  plt.savefig(image_str, format=image_format)
C:\anaconda3\envs\tf_nlp\lib\site-packages\pandas_profiling\visualisation\utils.py:66: UserWarning: Glyph 54644 (\N{HANGUL SYLLABLE HAE}) missing from current 

C:\anaconda3\envs\tf_nlp\lib\site-packages\pandas_profiling\visualisation\utils.py:66: UserWarning: Glyph 50668 (\N{HANGUL SYLLABLE YEO}) missing from current font.
  plt.savefig(image_str, format=image_format)
C:\anaconda3\envs\tf_nlp\lib\site-packages\pandas_profiling\visualisation\utils.py:66: UserWarning: Glyph 49888 (\N{HANGUL SYLLABLE SIN}) missing from current font.
  plt.savefig(image_str, format=image_format)
C:\anaconda3\envs\tf_nlp\lib\site-packages\pandas_profiling\visualisation\utils.py:66: UserWarning: Glyph 44592 (\N{HANGUL SYLLABLE GI}) missing from current font.
  plt.savefig(image_str, format=image_format)
C:\anaconda3\envs\tf_nlp\lib\site-packages\pandas_profiling\visualisation\utils.py:66: UserWarning: Glyph 44036 (\N{HANGUL SYLLABLE GAN}) missing from current font.
  plt.savefig(image_str, format=image_format)
C:\anaconda3\envs\tf_nlp\lib\site-packages\pandas_profiling\visualisation\utils.py:66: UserWarning: Glyph 49892 (\N{HANGUL SYLLABLE SIL}) missing from curren

C:\anaconda3\envs\tf_nlp\lib\site-packages\pandas_profiling\visualisation\utils.py:66: UserWarning: Glyph 50668 (\N{HANGUL SYLLABLE YEO}) missing from current font.
  plt.savefig(image_str, format=image_format)
C:\anaconda3\envs\tf_nlp\lib\site-packages\pandas_profiling\visualisation\utils.py:66: UserWarning: Glyph 49888 (\N{HANGUL SYLLABLE SIN}) missing from current font.
  plt.savefig(image_str, format=image_format)
C:\anaconda3\envs\tf_nlp\lib\site-packages\pandas_profiling\visualisation\utils.py:66: UserWarning: Glyph 44552 (\N{HANGUL SYLLABLE GEUM}) missing from current font.
  plt.savefig(image_str, format=image_format)
C:\anaconda3\envs\tf_nlp\lib\site-packages\pandas_profiling\visualisation\utils.py:66: UserWarning: Glyph 50529 (\N{HANGUL SYLLABLE AEG}) missing from current font.
  plt.savefig(image_str, format=image_format)
C:\anaconda3\envs\tf_nlp\lib\site-packages\pandas_profiling\visualisation\utils.py:66: UserWarning: Glyph 49892 (\N{HANGUL SYLLABLE SIL}) missing from curr

...

C:\anaconda3\envs\tf_nlp\lib\site-packages\pandas_profiling\visualisation\utils.py:66: UserWarning: Glyph 44592 (\N{HANGUL SYLLABLE GI}) missing from current font.
  plt.savefig(image_str, format=image_format)
C:\anaconda3\envs\tf_nlp\lib\site-packages\pandas_profiling\visualisation\utils.py:66: UserWarning: Glyph 51456 (\N{HANGUL SYLLABLE JUN}) missing from current font.
  plt.savefig(image_str, format=image_format)
C:\anaconda3\envs\tf_nlp\lib\site-packages\pandas_profiling\visualisation\utils.py:66: UserWarning: Glyph 51068 (\N{HANGUL SYLLABLE IL}) missing from current font.
  plt.savefig(image_str, format=image_format)
C:\anaconda3\envs\tf_nlp\lib\site-packages\pandas_profiling\visualisation\utils.py:66: UserWarning: Glyph 51088 (\N{HANGUL SYLLABLE JA}) missing from current font.
  plt.savefig(image_str, format=image_format)
C:\anaconda3\envs\tf_nlp\lib\site-packages\pandas_profiling\visualisation\utils.py:66: UserWarning: Glyph 49892 (\N{HANGUL SYLLABLE SIL}) missing from current 

C:\anaconda3\envs\tf_nlp\lib\site-packages\pandas_profiling\visualisation\utils.py:66: UserWarning: Glyph 54665 (\N{HANGUL SYLLABLE HAENG}) missing from current font.
  plt.savefig(image_str, format=image_format)
C:\anaconda3\envs\tf_nlp\lib\site-packages\pandas_profiling\visualisation\utils.py:66: UserWarning: Glyph 49324 (\N{HANGUL SYLLABLE SA}) missing from current font.
  plt.savefig(image_str, format=image_format)
C:\anaconda3\envs\tf_nlp\lib\site-packages\pandas_profiling\visualisation\utils.py:66: UserWarning: Glyph 50976 (\N{HANGUL SYLLABLE YU}) missing from current font.
  plt.savefig(image_str, format=image_format)
C:\anaconda3\envs\tf_nlp\lib\site-packages\pandas_profiling\visualisation\utils.py:66: UserWarning: Glyph 53076 (\N{HANGUL SYLLABLE KO}) missing from current font.
  plt.savefig(image_str, format=image_format)
C:\anaconda3\envs\tf_nlp\lib\site-packages\pandas_profiling\visualisation\utils.py:66: UserWarning: Glyph 46300 (\N{HANGUL SYLLABLE DEU}) missing from curren

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

FileNotFoundError: [Errno 2] No such file or directory: '..\\output\\report_AUTO.html'

### 실행월별 신규취급건 통계를 뽑습니다.

In [24]:
monthly_new_disburse_df = dataprep_df_full[dataprep_df_full['실행사유코드명'] == '신규']
# 피봇테이블 만들기
pv_monthly_disburse = pd.crosstab(monthly_new_disburse_df['VIN_M'], monthly_new_disburse_df['상품구분'], values=monthly_new_disburse_df['여신금액']/1000000, aggfunc='sum').round(0)

In [25]:
pv_monthly_disburse

상품구분,AUTO,기타,네이버사업자대출,데이터금융_기타,스탁론,임대보증금,투자금융 등
VIN_M,,,,,,,
199909,NaN,NaN,NaN,NaN,NaN,NaN,788.0
200801,NaN,NaN,NaN,NaN,NaN,NaN,20317.0
201204,NaN,NaN,NaN,NaN,NaN,NaN,2073.0
201601,NaN,NaN,NaN,NaN,NaN,NaN,16000.0
201607,NaN,NaN,NaN,NaN,NaN,NaN,20000.0
...,...,...,...,...,...,...,...
202108,31678.0,NaN,7851.0,695.0,20120.0,26749.0,21055.0
202109,23235.0,NaN,5881.0,923.0,12357.0,21039.0,55372.0
202110,20203.0,NaN,5323.0,1164.0,8987.0,24331.0,72608.0


In [30]:
import matplotlib.pyplot as plt

plt.figure(figsize=(16,9), dpi= 80)
pv_monthly_disburse.plot(kind="bar",stacked=True,figsize=(10,8))
plt.legend(loc="lower left",bbox_to_anchor=(0.8,1.0))
plt.savefig('monthly_disbursed.jpg')


C:\Users\C07037\AppData\Local\Temp/ipykernel_8412/3674063309.py:6: UserWarning: Glyph 44592 (\N{HANGUL SYLLABLE GI}) missing from current font.
  plt.savefig('monthly_disbursed.jpg')
C:\Users\C07037\AppData\Local\Temp/ipykernel_8412/3674063309.py:6: UserWarning: Glyph 53440 (\N{HANGUL SYLLABLE TA}) missing from current font.
  plt.savefig('monthly_disbursed.jpg')
C:\Users\C07037\AppData\Local\Temp/ipykernel_8412/3674063309.py:6: UserWarning: Glyph 45348 (\N{HANGUL SYLLABLE NE}) missing from current font.
  plt.savefig('monthly_disbursed.jpg')
C:\Users\C07037\AppData\Local\Temp/ipykernel_8412/3674063309.py:6: UserWarning: Glyph 51060 (\N{HANGUL SYLLABLE I}) missing from current font.
  plt.savefig('monthly_disbursed.jpg')
C:\Users\C07037\AppData\Local\Temp/ipykernel_8412/3674063309.py:6: UserWarning: Glyph 48260 (\N{HANGUL SYLLABLE BEO}) missing from current font.
  plt.savefig('monthly_disbursed.jpg')
C:\Users\C07037\AppData\Local\Temp/ipykernel_8412/3674063309.py:6: UserWarning: Glyph

### 실행월별 신규취급건 통계를 뽑습니다.

In [32]:
pv_monthly_disburse_avg = pd.crosstab(monthly_new_disburse_df['VIN_M'], monthly_new_disburse_df['상품구분'], values=monthly_new_disburse_df['여신금액']/1000, aggfunc='mean').round(0)

plt.figure(figsize=(16,9), dpi= 80)
pv_monthly_disburse_avg.plot(kind="bar",stacked=True,figsize=(10,8))
plt.legend(loc="lower left",bbox_to_anchor=(0.8,1.0))
plt.savefig('monthly_disbursed_average.jpg')

C:\Users\C07037\AppData\Local\Temp/ipykernel_8412/3922832042.py:6: UserWarning: Glyph 44592 (\N{HANGUL SYLLABLE GI}) missing from current font.
  plt.savefig('monthly_disbursed_average.jpg')
C:\Users\C07037\AppData\Local\Temp/ipykernel_8412/3922832042.py:6: UserWarning: Glyph 53440 (\N{HANGUL SYLLABLE TA}) missing from current font.
  plt.savefig('monthly_disbursed_average.jpg')
C:\Users\C07037\AppData\Local\Temp/ipykernel_8412/3922832042.py:6: UserWarning: Glyph 45348 (\N{HANGUL SYLLABLE NE}) missing from current font.
  plt.savefig('monthly_disbursed_average.jpg')
C:\Users\C07037\AppData\Local\Temp/ipykernel_8412/3922832042.py:6: UserWarning: Glyph 51060 (\N{HANGUL SYLLABLE I}) missing from current font.
  plt.savefig('monthly_disbursed_average.jpg')
C:\Users\C07037\AppData\Local\Temp/ipykernel_8412/3922832042.py:6: UserWarning: Glyph 48260 (\N{HANGUL SYLLABLE BEO}) missing from current font.
  plt.savefig('monthly_disbursed_average.jpg')
C:\Users\C07037\AppData\Local\Temp/ipykernel_

### 월별 신규고객수 추이

In [33]:
data_df_full = dataprep_df_full.copy()
data_df_full = data_df_full[['고객번호', 'VIN_M', '실행사유코드명', '상품구분', '실행일자']]
data_df_full = data_df_full[data_df_full['실행사유코드명'] == '신규']
data_df_full = data_df_full.sort_values('실행일자')

data_df_new = data_df_full.drop_duplicates(subset = '고객번호', keep='first').reset_index(drop=True)

pv_monthly_new_cus = pd.crosstab(data_df_new['VIN_M'], data_df_new['상품구분'], values=data_df_new['고객번호'], aggfunc='count')

plt.figure(figsize=(16,9), dpi= 80)
pv_monthly_disburse_avg.plot(kind="bar",stacked=True,figsize=(10,8))
plt.legend(loc="lower left",bbox_to_anchor=(0.8,1.0))
plt.savefig('monthly_new_customer.jpg')

C:\Users\C07037\AppData\Local\Temp/ipykernel_8412/760588847.py:13: UserWarning: Glyph 44592 (\N{HANGUL SYLLABLE GI}) missing from current font.
  plt.savefig('monthly_disbursed_average.jpg')
C:\Users\C07037\AppData\Local\Temp/ipykernel_8412/760588847.py:13: UserWarning: Glyph 53440 (\N{HANGUL SYLLABLE TA}) missing from current font.
  plt.savefig('monthly_disbursed_average.jpg')
C:\Users\C07037\AppData\Local\Temp/ipykernel_8412/760588847.py:13: UserWarning: Glyph 45348 (\N{HANGUL SYLLABLE NE}) missing from current font.
  plt.savefig('monthly_disbursed_average.jpg')
C:\Users\C07037\AppData\Local\Temp/ipykernel_8412/760588847.py:13: UserWarning: Glyph 51060 (\N{HANGUL SYLLABLE I}) missing from current font.
  plt.savefig('monthly_disbursed_average.jpg')
C:\Users\C07037\AppData\Local\Temp/ipykernel_8412/760588847.py:13: UserWarning: Glyph 48260 (\N{HANGUL SYLLABLE BEO}) missing from current font.
  plt.savefig('monthly_disbursed_average.jpg')
C:\Users\C07037\AppData\Local\Temp/ipykernel_